In [1]:
# load list at /home/zeyuan.yin/My-Adv/TransferAttack_sync/not_correct_list_label.pkl

import os
import pickle
import numpy as np
import pandas as pd


not_correct_list_label = pickle.load(open('/home/zeyuan.yin/My-Adv/TransferAttack_sync/not_correct_list_label.pkl', 'rb'))
not_correct_list_name = pickle.load(open('/home/zeyuan.yin/My-Adv/TransferAttack_sync/not_correct_list_name.pkl', 'rb'))

print(len(not_correct_list_label))
print(len(not_correct_list_name))

print(not_correct_list_label)
print(not_correct_list_name[:10])


10
10
[(58,), (269,), (595,), (79,), (369,), (596,), (837,), (248,), (664,), (572,)]
[('ILSVRC2012_val_00002836.JPEG',), ('ILSVRC2012_val_00000912.JPEG',), ('ILSVRC2012_val_00001840.JPEG',), ('ILSVRC2012_val_00003981.JPEG',), ('ILSVRC2012_val_00002857.JPEG',), ('ILSVRC2012_val_00002588.JPEG',), ('ILSVRC2012_val_00005437.JPEG',), ('ILSVRC2012_val_00002066.JPEG',), ('ILSVRC2012_val_00004758.JPEG',), ('ILSVRC2012_val_00004591.JPEG',)]


In [56]:
# step 1. remove the images in not_correct_list_name from data/

dir_path = '/home/zeyuan.yin/My-Adv/data/images'
# for name in not_correct_list_name:
#     name = name[0]
#     os.remove(os.path.join(dir_path, name))

# check the number of images in data/
print(len(os.listdir(dir_path)))

1000


In [4]:
# load pd in /home/zeyuan.yin/My-Adv/imagenet-check/cls_results_all_true_val.csv

df = pd.read_csv('/home/zeyuan.yin/My-Adv/imagenet-check/cls_results_all_true_val.csv', header=0)
print(df.head())

                           name  label
0  ILSVRC2012_val_00006697.JPEG      0
1  ILSVRC2012_val_00007197.JPEG      0
2  ILSVRC2012_val_00009111.JPEG      0
3  ILSVRC2012_val_00009191.JPEG      0
4  ILSVRC2012_val_00009346.JPEG      0


In [13]:
add_filename = []
for label in not_correct_list_label:
    label_idx = label[0]

    # select all name in the label_idx class
    name = df[df['label'] == label_idx]['name']
    # print(len(name))
    # print(name.iloc[0])
    if len(name) == 0:
        print('no name in this class')
        print(label_idx)
    else:
        add_filename.append(name.iloc[0])

print(len(add_filename))

no name in this class
638


In [18]:
# add an image in 638
# ILSVRC2012_val_00028738.JPEG,638,False,True,True,True,True,True,True,True

name = 'ILSVRC2012_val_00028738.JPEG'
add_filename.append(name)

print(len(add_filename))

# step 2. add the images in add_filename to data/

for name in add_filename:
    os.system('cp /l/users/zeyuan.yin/raw_val/' + name + ' /home/zeyuan.yin/My-Adv/data/images/' + name)

# check the number of images in data/
print(len(os.listdir(dir_path)))

165


1000


In [37]:
# gen labels.csv
# filename,label
# ILSVRC2012_val_00013716.JPEG,0
# ILSVRC2012_val_00020785.JPEG,1

# load pd in /home/zeyuan.yin/My-Adv/imagenet-check/cls_results_val.csv

df = pd.read_csv('/home/zeyuan.yin/My-Adv/imagenet-check/cls_results_val.csv', header=0)
# print(df.head())

# load all filename under /home/zeyuan.yin/My-Adv/data/images
all_filename = os.listdir(dir_path)

# extract df for only name in add_filename
df = df[df['name'].isin(all_filename)]
print(len(df))
# print(df.head(5))

# only save the name and label in df
df = df[['name', 'label']]
print(df.head(5))

# save to /home/zeyuan.yin/My-Adv/data/labels_new.csv
df.to_csv('/home/zeyuan.yin/My-Adv/data/labels_new.csv', index=False)

1000
                             name  label
14   ILSVRC2012_val_00013716.JPEG      0
74   ILSVRC2012_val_00020785.JPEG      1
138  ILSVRC2012_val_00037998.JPEG      2
177  ILSVRC2012_val_00029568.JPEG      3
233  ILSVRC2012_val_00031424.JPEG      4


In [54]:
## generate random target label (int)

df = pd.read_csv('/home/zeyuan.yin/My-Adv/data/labels_new.csv', header=0)

print(df.dtypes)
for label in df['label']:
    targeted_label = np.random.randint(0, 1000)
    assert targeted_label != label, 'retry'
    df.loc[df['label'] == label, 'targeted_label'] = str(int(targeted_label))


print(df.head(5))

df.to_csv('/home/zeyuan.yin/My-Adv/data/labels_targeted_new.csv', index=False)

name     object
label     int64
dtype: object
                           name  label targeted_label
0  ILSVRC2012_val_00013716.JPEG      0            790
1  ILSVRC2012_val_00020785.JPEG      1            262
2  ILSVRC2012_val_00037998.JPEG      2            562
3  ILSVRC2012_val_00029568.JPEG      3            438
4  ILSVRC2012_val_00031424.JPEG      4            661
